# Tutorial 4: detect novel cells in the test data via CANAL

In [1]:
import os
os.chdir('/data/wanh/CANAL/') 
import sys
sys.path.append('/data/wanh/CANAL/')
import argparse
from model import *
import time
import resource

## Construct and run the CANAL model for three stages

In [2]:
experiments = 'Pancreas_novel_cells'
seed = 1
with open(f'./data/Pancreas/Pancreas_highly_gene_idx.csv') as csvfile:
      spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
      highly_variable_idx = []
      for row in spamreader:
            highly_variable_idx.append(row[0])

highly_variable_idx = [int(i) for i in highly_variable_idx]
highly_variable_idx = np.array(highly_variable_idx)

In [3]:
CANAL = CANAL_model(gpu_option = '3')

In [4]:
dataset_stage_1 = "Muraro"
data_path_stage_1 = './data/Pancreas/{}_train.h5ad'.format(dataset_stage_1)
adata_stage_1 = sc.read_h5ad(data_path_stage_1)
adata_stage_1 = adata_stage_1[np.array(adata_stage_1.obs['cell_type1']) != 'alpha']
cell_type_stage_1 = adata_stage_1.obs['cell_type1']
print(adata_stage_1) 
print(np.unique(np.array(cell_type_stage_1), return_counts=True))

View of AnnData object with n_obs × n_vars = 985 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['acinar', 'beta', 'delta', 'ductal', 'gamma', 'mesenchymal'],
      dtype=object), array([168, 352, 153, 177,  76,  59]))


In [5]:
CANAL.train(experiments = experiments, pre_dataset = "None", dataset = dataset_stage_1, 
            adata = adata_stage_1, cell_type = cell_type_stage_1, current_stage = 1, 
            is_final_stage = False, ckpt_dir = './ckpts/', rehearsal_size = 1000, 
            highly_variable_idx=highly_variable_idx, SEED = seed)

current data: View of AnnData object with n_obs × n_vars = 985 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['acinar', 'beta', 'delta', 'ductal', 'gamma', 'mesenchymal'],
      dtype=object), array([168, 352, 153, 177,  76,  59]))
model constructing begin!



/data/wanh/CANAL/performer_pytorch/performer_pytorch.py:115: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2497.)
  q, r = torch.qr(unstructured_block.cpu(), some = True)
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


model constructing finished!

label train: [0 1 2 3 4 5] 6
label val: [0 1 2 3 4 5] 6
  ==  Begin finetuning: | Initial stage | Current stage: 1 | CANAL | Dataset: Pancreas_novel_cells Muraro ==
    ==  Epoch: 1 | Classification Loss: 1.795873 | Representation DL Loss: 0.000000  | Accuracy: 13.9031%  ==
    ==  Epoch: 1 | Validation CLS Loss: 1.784394 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.057665  == | Current accuracy: 0.209184  ==
[[ 0  0  0 40  0  0]
 [ 0  0  0 67  0  0]
 [ 0  0  0 26  0  0]
 [ 0  0  0 41  0  0]
 [ 0  0  0 11  0  0]
 [ 0  0  0 11  0  0]]
Patience: 0 / 10
    ==  Epoch: 2 | Classification Loss: 1.667545 | Representation DL Loss: 0.000000  | Accuracy: 31.7602%  ==
    ==  Epoch: 2 | Validation CLS Loss: 1.639460 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.083969  == | Current accuracy: 0.336735  ==
[[ 0 47  0  0  0  0]
 [ 0 66  0  0  0  0]
 [ 0 29  0  0  0  0]
 [ 0 26  0  0  0  0]
 [ 0 13  0  0  0  0]
 [ 0 15  0  0  0  0]]
Patience

    ==  Epoch: 21 | Classification Loss: 0.512222 | Representation DL Loss: 0.000000  | Accuracy: 97.3214%  ==
    ==  Epoch: 21 | Validation CLS Loss: 3.108909 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.903065  == | Current accuracy: 0.908163  ==
[[23  0  0  0  0  0]
 [ 0 68  2  3  1  0]
 [ 0  0 29  0  0  0]
 [ 0  4  0 34  0  0]
 [ 0  0  8  0 12  0]
 [ 0  0  0  0  0 12]]
Patience: 11 / 10


/data/wanh/CANAL/model.py:106: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_i.obs['celltype'] = current_label_dict[i]
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],




example bank after updating:
AnnData object with n_obs × n_vars = 786 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype', 'rank', 'stage'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'


cell type composition of this example bank:
(array(['acinar', 'beta', 'delta', 'ductal', 'gamma', 'mesenchymal'],
      dtype=object), array([166, 166, 153, 166,  76,  59]))


dataset composition from each stage of this example bank:
(array([1]), array([786]))




In [6]:
dataset_stage_2 = "Enge"
data_path_stage_2 = './data/Pancreas/{}_train.h5ad'.format(dataset_stage_2)
adata_stage_2 = sc.read_h5ad(data_path_stage_2)
adata_stage_2 = adata_stage_2[np.array(adata_stage_2.obs['cell_type1']) != 'alpha']
cell_type_stage_2 = adata_stage_2.obs['cell_type1']
print(adata_stage_2)  
print(np.unique(np.array(cell_type_stage_2), return_counts=True))

View of AnnData object with n_obs × n_vars = 998 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['acinar', 'beta', 'delta', 'ductal', 'mesenchymal'], dtype=object), array([332, 267,  55, 300,  44]))


In [7]:
CANAL.train(experiments = experiments, pre_dataset = dataset_stage_1, dataset = dataset_stage_2, 
            adata = adata_stage_2, cell_type = cell_type_stage_2, current_stage = 2, 
            is_final_stage = False, ckpt_dir='./ckpts/', rehearsal_size=1000, SEED = seed)

current data: View of AnnData object with n_obs × n_vars = 998 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['acinar', 'beta', 'delta', 'ductal', 'mesenchymal'], dtype=object), array([332, 267,  55, 300,  44]))
model constructing begin!

new cell types: []
example bank for experience replay: AnnData object with n_obs × n_vars = 786 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype', 'rank', 'stage'
    var: 'highly_variable', 

/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


  ==  Begin finetuning: | Incrmental stage | Current stage: 2 | CANAL | Dataset: Pancreas_novel_cells Enge  ==
    ==  Epoch: 1 | Classification Loss: 0.655114 | Representation DL Loss: 0.034605  | Accuracy: 91.9378%  ==
    ==  Epoch: 1 | Validation CLS Loss: 3.109473 | Validation Representation DL Loss: 0.080488 | F1 Score: 0.933107  == | Current accuracy: 0.957143  ==
[[105   0   0   0   0   0]
 [  0  70   3   0   5   0]
 [  0   1  30   0   0   0]
 [  0   0   0  91   4   0]
 [  0   0   0   0  18   0]
 [  0   1   0   1   0  21]]
Patience: 0 / 10
    ==  Epoch: 2 | Classification Loss: 0.584491 | Representation DL Loss: 0.159628  | Accuracy: 94.8373%  ==
    ==  Epoch: 2 | Validation CLS Loss: 3.223272 | Validation Representation DL Loss: 0.177585 | F1 Score: 0.943006  == | Current accuracy: 0.960000  ==
[[87  0  0  0  0  0]
 [ 0 82  0  0  0  0]
 [ 0  4 37  0  0  0]
 [ 0  3  1 95  0  0]
 [ 0  0  3  0 13  0]
 [ 0  1  0  2  0 22]]
Patience: 0 / 10
    ==  Epoch: 3 | Classification Loss:

    ==  Epoch: 20 | Classification Loss: 0.477590 | Representation DL Loss: 0.165486  | Accuracy: 98.7270%  ==
    ==  Epoch: 20 | Validation CLS Loss: 3.339000 | Validation Representation DL Loss: 0.251615 | F1 Score: 0.954600  == | Current accuracy: 0.965714  ==
[[94  0  0  0  0  0]
 [ 0 85  0  0  0  0]
 [ 0 10 26  0  0  0]
 [ 0  0  0 98  0  0]
 [ 0  0  0  0 14  0]
 [ 0  0  0  2  0 21]]
Patience: 5 / 10
    ==  Epoch: 21 | Classification Loss: 0.473712 | Representation DL Loss: 0.188792  | Accuracy: 99.1513%  ==
    ==  Epoch: 21 | Validation CLS Loss: 2.950457 | Validation Representation DL Loss: 0.243565 | F1 Score: 0.941742  == | Current accuracy: 0.951429  ==
[[98  0  0  0  0  0]
 [ 0 71 13  0  0  0]
 [ 0  2 33  0  0  0]
 [ 0  0  0 93  0  0]
 [ 0  0  1  0 15  0]
 [ 0  0  0  1  0 23]]
Patience: 6 / 10
    ==  Epoch: 22 | Classification Loss: 0.475351 | Representation DL Loss: 0.193461  | Accuracy: 99.2221%  ==
    ==  Epoch: 22 | Validation CLS Loss: 3.472855 | Validation Represen

/data/wanh/CANAL/model.py:106: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_i.obs['celltype'] = current_label_dict[i]
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],




example bank after updating:
AnnData object with n_obs × n_vars = 843 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype', 'rank', 'stage'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'


cell type composition of this example bank:
(array(['acinar', 'beta', 'delta', 'ductal', 'gamma', 'mesenchymal'],
      dtype=object), array([166, 166, 166, 166,  76, 103]))


dataset composition from each stage of this example bank:
(array([1, 2]), array([495, 348]))




In [8]:
dataset_stage_3 = "Baron_human"
data_path_stage_3 = './data/Pancreas/{}_train.h5ad'.format(dataset_stage_3)
adata_stage_3 = sc.read_h5ad(data_path_stage_3)
adata_stage_3 = adata_stage_3[np.array(adata_stage_3.obs['cell_type1']) != 'alpha']
cell_type_stage_3 = adata_stage_3.obs['cell_type1']
print(adata_stage_3)  
print(np.unique(np.array(cell_type_stage_3), return_counts=True))

View of AnnData object with n_obs × n_vars = 5775 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['acinar', 'activated_stellate', 'beta', 'delta', 'ductal',
       'endothelial', 'gamma', 'quiescent_stellate'], dtype=object), array([ 908,  268, 2367,  570, 1016,  241,  243,  162]))


In [9]:
CANAL.train(experiments = experiments, pre_dataset = dataset_stage_2, dataset = dataset_stage_3, 
            adata = adata_stage_3, cell_type = cell_type_stage_3, current_stage = 3, is_final_stage = True,   
            ckpt_dir='./ckpts/', rehearsal_size=1000, SEED = seed)

current data: View of AnnData object with n_obs × n_vars = 5775 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['acinar', 'activated_stellate', 'beta', 'delta', 'ductal',
       'endothelial', 'gamma', 'quiescent_stellate'], dtype=object), array([ 908,  268, 2367,  570, 1016,  241,  243,  162]))
model constructing begin!

new cell types: ['activated_stellate', 'endothelial', 'quiescent_stellate']
example bank for experience replay: AnnData object with n_obs × n_vars = 843 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libs

/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


    ==  Epoch: 1 | Classification Loss: 1.044161 | Representation DL Loss: 0.089578  | Accuracy: 85.2797%  ==
    ==  Epoch: 1 | Validation CLS Loss: 2.562565 | Validation Representation DL Loss: 0.173877 | F1 Score: 0.530131  == | Current accuracy: 0.835106  ==
[[215   2   0   0   0   0   0   0   0]
 [  3 466   9   2   0   0   0   0   0]
 [  0  16 150   0   0   0   0   0   0]
 [ 33   2   4 201   0   0   0   0   0]
 [  2  13   5   0  45   0   0   0   0]
 [  0   0   0   0   0  22   0   0   0]
 [  0   3   3   0   0  42   0   0   0]
 [  2  11   1  17   1  11   0   0   0]
 [  0   9   0   0   0  26   0   0   0]]
Patience: 0 / 10
    ==  Epoch: 2 | Classification Loss: 0.832789 | Representation DL Loss: 0.456191  | Accuracy: 86.7536%  ==
    ==  Epoch: 2 | Validation CLS Loss: 3.191035 | Validation Representation DL Loss: 0.367353 | F1 Score: 0.800855  == | Current accuracy: 0.925532  ==
[[220   0   0   0   0   0   0   0   0]
 [  6 484  11   0   2   0   0   0   0]
 [  0   5 146   0   0   0  

    ==  Epoch: 14 | Classification Loss: 0.527888 | Representation DL Loss: 0.287770  | Accuracy: 99.2252%  ==
    ==  Epoch: 14 | Validation CLS Loss: 3.247332 | Validation Representation DL Loss: 0.289546 | F1 Score: 0.975222  == | Current accuracy: 0.974924  ==
[[195   1   0   2   0   0   0   0   1]
 [  3 489   4   0   1   0   0   0   0]
 [  0  11 151   0   0   0   0   0   0]
 [  2   0   0 228   0   0   0   0   0]
 [  1   2   0   0  52   0   0   0   0]
 [  0   0   0   0   0  17   0   0   0]
 [  0   0   1   0   0   0  50   0   0]
 [  0   2   0   0   0   0   0  70   1]
 [  0   1   0   0   0   0   0   0  31]]
Patience: 0 / 10
    ==  Epoch: 15 | Classification Loss: 0.531741 | Representation DL Loss: 0.278232  | Accuracy: 98.8662%  ==
    ==  Epoch: 15 | Validation CLS Loss: 3.454890 | Validation Representation DL Loss: 0.279549 | F1 Score: 0.958301  == | Current accuracy: 0.971884  ==
[[217   1   0   3   0   0   0   0   1]
 [  2 493   1   0   0   0   0   0   0]
 [  0   8 126   0   0  

    ==  Epoch: 27 | Classification Loss: 0.519523 | Representation DL Loss: 0.284533  | Accuracy: 99.3953%  ==
    ==  Epoch: 27 | Validation CLS Loss: 3.534639 | Validation Representation DL Loss: 0.289244 | F1 Score: 0.979500  == | Current accuracy: 0.981003  ==
[[202   0   0   2   0   0   0   0   2]
 [  6 490   1   0   0   0   0   0   0]
 [  0   2 162   0   0   0   0   0   0]
 [  5   0   0 232   0   0   0   0   0]
 [  1   0   5   0  54   0   0   0   0]
 [  0   0   0   0   0  18   0   0   0]
 [  0   0   0   0   0   0  47   0   0]
 [  0   0   0   0   0   0   0  44   1]
 [  0   0   0   0   0   0   0   0  42]]
Patience: 0 / 10
    ==  Epoch: 28 | Classification Loss: 0.519246 | Representation DL Loss: 0.275541  | Accuracy: 99.4520%  ==
    ==  Epoch: 28 | Validation CLS Loss: 3.718932 | Validation Representation DL Loss: 0.259004 | F1 Score: 0.975520  == | Current accuracy: 0.979483  ==
[[225   2   0   2   0   0   0   0   1]
 [  1 492   5   0   1   0   0   0   0]
 [  0   4 144   0   0  

## predict cell types of the test data
### predicted unknown cells will be annotated as "Unassigned"

In [10]:
dataset_test = sc.read_h5ad("./data/Pancreas/Pancreas_test.h5ad")
print(dataset_test)
pred_cell_type = CANAL.predict(adata_predict = dataset_test, ckpt_dir = './ckpts/', experiments = experiments,
                               stage_num=3, dataset = dataset_stage_3, novel=True)

AnnData object with n_obs × n_vars = 2000 × 1000
    obs: 'donor', 'cell_type1', 'library', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__', 'baron_human_donor', 'age', 'gender', 'enge_donor', 'batch', 'muraro_donor', 'celltype', 'batch2'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    ==  Begin predicting after 3 fine-tuning stages: | Experiments: Pancreas_novel_cells ==
Annotation: ['acinar' 'beta' 'delta' 'ductal' 'gamma' 'mesenchymal'
 'activated_stellate' 'endothelial' 'quiescent_stellate'] 



## evaluate the performance of cell-type annotation and novel cell detection

In [11]:
true_celltype = np.array(dataset_test.obs['cell_type1']) 
CANAL.evaluation(pred_cell_type=pred_cell_type, true_celltype=true_celltype, novel_celltype='alpha')

Begin evaluation!

  ==  H-score: 0.824017 ==|== Predict total accuracy: 0.841000  ==

  ==  Predict known accuracy: 0.887336  ==|== Predict unknown accuracy: 0.769133  == 

